# Data Analysis

# Exploratory Data Analysis (EDA)

**Objective:**  
In this notebook, we explore the dataset from the Kaggle competition [House Prices: Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques) to gain insights into the structure, distribution, and quality of the data. Our goal is to better understand which variables might influence housing prices and how they relate to the target variable `SalePrice`.

---

## Key Goals:

1. **Understand the data composition**
   - Dimensions of the training dataset
   - Data types (numerical, categorical, ordinal)
   - Summary statistics and initial observations

2. **Handle missing values**
   - Identify features with missing data
   - Assess impact and potential imputation strategies

3. **Univariate and bivariate analysis**
   - Distribution and skewness of features
   - Correlation with the target variable (`SalePrice`)
   - Visual exploration using histograms, boxplots, heatmaps, and scatter plots

4. **Detect outliers and anomalies**
   - Visual outlier detection
   - Considerations for removing or transforming extreme values

---

**Outcome:**  
This analysis will guide feature engineering decisions and highlight important variables that could improve model performance.

In [1]:
# Import Libraries
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import hvplot.polars
import hvplot.pandas
import numpy as np
from bokeh.models import NumeralTickFormatter
import holoviews as hv
from bokeh.models import NumeralTickFormatter
hv.extension('bokeh')

In [2]:
# Load the dataset, handle missing values, drop Id column and get basic information
data = pl.read_csv('data/train.csv', null_values="NA")
data = data.drop('Id')
print(f'The dataset contains {data.shape[0]} rows and {data.shape[1]} columns')
data.head()

The dataset contains 1460 rows and 80 columns


MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,i64,str,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,i64,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
60,"""RL""",65,8450,"""Pave""",null,"""Reg""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2003,2003,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""",196,"""Gd""","""TA""","""PConc""","""Gd""","""TA""","""No""","""GLQ""",706,"""Unf""",0,150,…,854,0,1710,1,0,2,1,3,1,"""Gd""",8,"""Typ""",0,null,"""Attchd""",2003,"""RFn""",2,548,"""TA""","""TA""","""Y""",0,61,0,0,0,0,null,null,null,0,2,2008,"""WD""","""Normal""",208500
20,"""RL""",80,9600,"""Pave""",null,"""Reg""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""Veenker""","""Feedr""","""Norm""","""1Fam""","""1Story""",6,8,1976,1976,"""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""",0,"""TA""","""TA""","""CBlock""","""Gd""","""TA""","""Gd""","""ALQ""",978,"""Unf""",0,284,…,0,0,1262,0,1,2,0,3,1,"""TA""",6,"""Typ""",1,"""TA""","""Attchd""",1976,"""RFn""",2,460,"""TA""","""TA""","""Y""",298,0,0,0,0,0,null,null,null,0,5,2007,"""WD""","""Normal""",181500
60,"""RL""",68,11250,"""Pave""",null,"""IR1""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2001,2002,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""",162,"""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Mn""","""GLQ""",486,"""Unf""",0,434,…,866,0,1786,1,0,2,1,3,1,"""Gd""",6,"""Typ""",1,"""TA""","""Attchd""",2001,"""RFn""",2,608,"""TA""","""TA""","""Y""",0,42,0,0,0,0,null,null,null,0,9,2008,"""WD""","""Normal""",223500
70,"""RL""",60,9550,"""Pave""",null,"""IR1""","""Lvl""","""AllPub""","""Corner""","""Gtl""","""Crawfor""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,1915,1970,"""Gable""","""CompShg""","""Wd Sdng""","""Wd Shng""","""None""",0,"""TA""","""TA""","""BrkTil""","""TA""","""Gd""","""No""","""ALQ""",216,"""Unf""",0,540,…,756,0,1717,1,0,1,0,3,1,"""Gd""",7,"""Typ""",1,"""Gd""","""Detchd""",1998,"""Unf""",3,642,"""TA""","""TA""","""Y""",0,35,272,0,0,0,null,null,null,0,2,2006,"""WD""","""Abnorml""",140000
60,"""RL""",84,14260,"""Pave""",null,"""IR1""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""NoRidge""","""Norm""","""Norm""","""1Fam""","""2Story""",8,5,2000,2000,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""",350,"""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Av""","""GLQ""",655,"""Unf""",0,490,…,1053,0,2198,1,0,2,1,4,1,"""Gd""",9,"""Typ""",1,"""TA""","""Attchd""",2000,"""RFn""",3,836,"""TA""","""TA""","""Y""",192,84,0,0,0,0,null,null,null,0,12,2008,"""WD""","""Normal""",250000


#### 1. Target Label

In [3]:
# Plot the distribution of our target label
prices = data.select('SalePrice')

plot = prices.hvplot.hist( # type: ignore
    bins=100,
    title='Sales Prices Distribution',
    xlabel='Sales Price ($)',
    ylabel='Frequency',
    xticks=np.arange(0, 800000, 100000),
    xlim=(0, 800000),
    ylim=(0, 100)
)
  
plot = plot.opts(show_grid=True, width=800,height=400, xformatter=NumeralTickFormatter(format="0,0"))
plot

:Histogram   [SalePrice]   (Count)

In [4]:
# From the distribution we can see that the prices are right skewed, we can use log transformation to make it be closer to a normal distribution
prices = prices.with_columns((pl.col('SalePrice') + 1).log().alias('LogScalePrice'))

log_plot = prices.hvplot.hist( # type: ignore
    'LogScalePrice',
    bins=100,
    title='Log Sales Prices Distribution',
    xlabel='Log Sales Price',
    ylabel='Frequency',
    xticks=np.arange(10, 20, 1),
)
log_plot = log_plot.opts(show_grid=True, width=800,height=400, xformatter=NumeralTickFormatter(format="0,0"))
log_plot

:Histogram   [LogScalePrice]   (Count)

#### 2. Variable types (categorical and numerical)

In [5]:
# Categorical and Numerical Features
categorical_features = data.select(pl.col(pl.String))
numerical_features   = data.select(pl.col(pl.Int64))

print(f'The dataset contains {categorical_features.shape[1]} categorical features and {numerical_features.shape[1]} numerical features')

The dataset contains 43 categorical features and 37 numerical features


#### 3. Missing Values

In [6]:
# Find all the columns with missing values
missing_val_columns = [column for column in data.columns if data[column].null_count() > 0]
print(f'The dataset contains {len(missing_val_columns)} columns with missing values: {missing_val_columns}')

# Calculate the percentage of missing values for each column and get the top 10 columns with the highest percentage of missing values
missing_val_percentage = data.select([pl.col(column).null_count() / data.height * 100 for column in missing_val_columns])
missing_val_percentage_sorted = missing_val_percentage.melt().sort("value", descending=True)
missing_val_percentage_sorted.head(10)

The dataset contains 19 columns with missing values: ['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


/var/folders/k5/22sywl_15k17yky84cqwzc_80000gn/T/ipykernel_63080/788216999.py:7: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  missing_val_percentage_sorted = missing_val_percentage.melt().sort("value", descending=True)


variable,value
str,f64
"""PoolQC""",99.520548
"""MiscFeature""",96.30137
"""Alley""",93.767123
"""Fence""",80.753425
"""FireplaceQu""",47.260274
"""LotFrontage""",17.739726
"""GarageType""",5.547945
"""GarageYrBlt""",5.547945
"""GarageFinish""",5.547945


In [7]:
# It makes sense that columns where we have more than 5% of missing values are not useful for our analysis and we can drop them
columns_to_drop = missing_val_percentage_sorted.filter(pl.col('value') > 10).select('variable').to_series().to_list()
data = data.drop(columns_to_drop)
print(f'The dataset now contains {data.height} rows and {data.width} columns after dropping the columns with more than 10% of missing values')

The dataset now contains 1460 rows and 74 columns after dropping the columns with more than 10% of missing values


#### 4. Numerical Variables Analysis

In [8]:
# Some of the interesting categorical features are the following:
numerical_features = data.select(pl.col(pl.Int64))
numerical_data = data.select(numerical_features)
numerical_data.head()

MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
60,8450,7,5,2003,2003,196,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003,2,548,0,61,0,0,0,0,0,2,2008,208500
20,9600,6,8,1976,1976,0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976,2,460,298,0,0,0,0,0,0,5,2007,181500
60,11250,7,5,2001,2002,162,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001,2,608,0,42,0,0,0,0,0,9,2008,223500
70,9550,7,5,1915,1970,0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998,3,642,0,35,272,0,0,0,0,2,2006,140000
60,14260,8,5,2000,2000,350,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000,3,836,192,84,0,0,0,0,0,12,2008,250000


In [9]:
# Some of the interesting numerical features are the Temporal features, which in the dataset are:
# 1. YearBuilt   : Original construction date
# 2. YearRemodAdd:  Remodel date (same as construction date if no remodeling or additions)
# 3. MoSold      :  Month Sold (MM)
# 4. YrSold      :  Year Sold
# 5. GarageYrBlt : Year garage was built
# The interesting part we can analyse is the relationship between the years built, remodeled and sold and the average sale price of the house

avg_price_by_year = data.group_by('YearBuilt').agg(pl.col('SalePrice').mean().alias('AvgSalePrice')).sort('YearBuilt')
year_plot = avg_price_by_year.hvplot.line(
    x='YearBuilt',
    y='AvgSalePrice',
    title='Average Sale Price by Year Built',
    xlabel='Year Built',
    ylabel='Average Sale Price ($)',
    xticks=np.arange(1870, 2015, 10),
)

year_plot = year_plot.opts(show_grid=True, width=800, height=400, yformatter=NumeralTickFormatter(format="0,0"))
year_plot

:Curve   [YearBuilt]   (AvgSalePrice)

In [10]:
avg_price_by_remodeling_year = data.group_by('YearRemodAdd').agg(pl.col('SalePrice').mean().alias('AvgSalePrice')).sort('YearRemodAdd')
remodel_plot = avg_price_by_remodeling_year.hvplot.line(
    x='YearRemodAdd',
    y='AvgSalePrice',
    title='Average Sale Price by Remodel Year',
    xlabel='Year Built',
    ylabel='Average Sale Price ($)',
    xticks=np.arange(1870, 2015, 5),
)

remodel_plot = remodel_plot.opts(show_grid=True,width=800, height=400, yformatter=NumeralTickFormatter(format="0,0"))
remodel_plot

:Curve   [YearRemodAdd]   (AvgSalePrice)

In [11]:
avg_price_by_year_sold = data.group_by('YrSold').agg(pl.col('SalePrice').mean().alias('AvgSalePrice')).sort('YrSold')
year_sold_plot = avg_price_by_year_sold.hvplot.line(
    x='YrSold',
    y='AvgSalePrice',
    title='Average Sale Price by Year Sold',
    xlabel='Year Built',
    ylabel='Average Sale Price ($)',
    xticks=np.arange(1870, 2015, 1)
    )

year_sold_plot = year_sold_plot.opts(show_grid=True, width=800, height=400, yformatter=NumeralTickFormatter(format="0,0"))
year_sold_plot

:Curve   [YrSold]   (AvgSalePrice)

In [12]:
# Next step is to find all the columns with discrete values that are not categorical features
discrete_features = [column for column in data.columns if data[column].dtype == pl.Int64 and data[column].n_unique() < 20]
print(f'The dataset contains: {len(discrete_features)} discrete features that are the following: {discrete_features}')

# These discrete variables tend to be qualifications (Qual) or grading scales (Cond), or refer to the number of rooms, 
# or units (FullBath, GarageCars), or indicate the area of the room (KitchenAbvGr). We expect higher prices, with bigger numbers.

The dataset contains: 15 discrete features that are the following: ['MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'PoolArea', 'MoSold', 'YrSold']


## Analyzing Discrete Features with Boxplots

In this section, we explore how various discrete numerical features influence the house sale price.  
A **boxplot** is used to visualize the distribution of sale prices across different values of each feature.  
This allows us to easily observe the spread, median, and presence of outliers for each category.

In [13]:
# For each one of the discrete features we are going to plot a boxplot 
discrete_features_with_target = discrete_features + ['SalePrice']
discrete_data = data.select(discrete_features_with_target)

for feature in discrete_features:
    sorted_data = discrete_data.sort(by=feature)
    display(
        sorted_data.hvplot.box(
            y='SalePrice', 
            by=feature, 
            xlabel=f'{feature}', 
            ylabel='Sale Price ($)', 
            title=f'Sale Price for Feature {feature}', 
            height=400, 
            width=800, 
            grid=True, 
            yformatter=NumeralTickFormatter(format="0,0")
            
        )
    )

:BoxWhisker   [MSSubClass]   (SalePrice)

:BoxWhisker   [OverallQual]   (SalePrice)

:BoxWhisker   [OverallCond]   (SalePrice)

:BoxWhisker   [BsmtFullBath]   (SalePrice)

:BoxWhisker   [BsmtHalfBath]   (SalePrice)

...

:BoxWhisker   [Fireplaces]   (SalePrice)

:BoxWhisker   [GarageCars]   (SalePrice)

:BoxWhisker   [PoolArea]   (SalePrice)

:BoxWhisker   [MoSold]   (SalePrice)

:BoxWhisker   [YrSold]   (SalePrice)

#### 5. Categorical Variables Analysis

In [14]:
# Some of the interesting categorical features are the following:
categorical_features = data.select(pl.col(pl.String)).columns
categorical_data = data.select(categorical_features)
categorical_data.head()

MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""RL""","""Pave""","""Reg""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""","""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""No""","""GLQ""","""Unf""","""GasA""","""Ex""","""Y""","""SBrkr""","""Gd""","""Typ""","""Attchd""","""RFn""","""TA""","""TA""","""Y""","""WD""","""Normal"""
"""RL""","""Pave""","""Reg""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""Veenker""","""Feedr""","""Norm""","""1Fam""","""1Story""","""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""TA""","""TA""","""CBlock""","""Gd""","""TA""","""Gd""","""ALQ""","""Unf""","""GasA""","""Ex""","""Y""","""SBrkr""","""TA""","""Typ""","""Attchd""","""RFn""","""TA""","""TA""","""Y""","""WD""","""Normal"""
"""RL""","""Pave""","""IR1""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""","""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Mn""","""GLQ""","""Unf""","""GasA""","""Ex""","""Y""","""SBrkr""","""Gd""","""Typ""","""Attchd""","""RFn""","""TA""","""TA""","""Y""","""WD""","""Normal"""
"""RL""","""Pave""","""IR1""","""Lvl""","""AllPub""","""Corner""","""Gtl""","""Crawfor""","""Norm""","""Norm""","""1Fam""","""2Story""","""Gable""","""CompShg""","""Wd Sdng""","""Wd Shng""","""None""","""TA""","""TA""","""BrkTil""","""TA""","""Gd""","""No""","""ALQ""","""Unf""","""GasA""","""Gd""","""Y""","""SBrkr""","""Gd""","""Typ""","""Detchd""","""Unf""","""TA""","""TA""","""Y""","""WD""","""Abnorml"""
"""RL""","""Pave""","""IR1""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""NoRidge""","""Norm""","""Norm""","""1Fam""","""2Story""","""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Av""","""GLQ""","""Unf""","""GasA""","""Ex""","""Y""","""SBrkr""","""Gd""","""Typ""","""Attchd""","""RFn""","""TA""","""TA""","""Y""","""WD""","""Normal"""


## Sorting Boxplots by Median Sale Price

To improve interpretability, we sort the x-axis categories based on the median sale price.  
This is achieved by calculating the median `SalePrice` for each unique value of the discrete feature,  
then converting the feature into a `pandas.Categorical` type with that order.  
This ensures that the boxplot displays the categories in ascending order of their typical sale price.

In [15]:
for feature in categorical_features:
    # Get value counts for each category
    counts = categorical_data.select(feature).to_pandas()[feature].value_counts().reset_index()
    counts.columns = [feature, 'Count']
    display(
        counts.hvplot.bar(
            x=feature,
            y='Count',
            xlabel=feature,
            ylabel='Count',
            title=f'Count of each category in {feature}',
            height=400,
            width=800,
            grid=True,
        )
    )

:Bars   [MSZoning]   (Count)

:Bars   [Street]   (Count)

:Bars   [LotShape]   (Count)

:Bars   [LandContour]   (Count)

:Bars   [Utilities]   (Count)

...

:Bars   [GarageQual]   (Count)

:Bars   [GarageCond]   (Count)

:Bars   [PavedDrive]   (Count)

:Bars   [SaleType]   (Count)

:Bars   [SaleCondition]   (Count)